# Create a heart-shape cutout

Create a slab with a complex cutout pattern.
By default, this notebook generates a structure of etching pattern:
- Two vertical side walls
- Two parallel channels at the top
- A deeper central cavity

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
2. Set structural parameters in cell 1.2 below
4. Click "Run" > "Run All" to create the structure
5. Scroll down to view the resulting structure from multiple angles

## Resulting structure:
<img src="https://i.imgur.com/j9YDJ1X.png">

## 1. Prepare the Environment
### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("")

### 1.2. Set up structure parameters

In [ ]:
# Slab parameters
MILLER_INDICES = (0, 0, 1)
THICKNESS = 10  # unit cell layers
VACUUM = 6.0  # Angstrom
XY_SUPERCELL_MATRIX = [[10, 0], [0, 10]]

# Heart shape parameters in crystal units
HEART_WIDTH = 0.75  # Width of heart relative to cell
HEART_HEIGHT = 0.8  # Height of heart relative to cell
NAME = "Diamond Heart Y-taper - C"

### 1.3. Create a condition on coordinates


In [ ]:


from typing import List
from mat3ra.made.tools.utils.coordinate import CoordinateCondition

class YTaperedHeartShapeCondition(CoordinateCondition):
    """
    Defines a heart-shaped structure with y-direction tapering applied.
    All dimensions are specified as fractions of the total size (0.0 to 1.0).
    """
    width: float = 0.75  # Heart width relative to cell size
    height: float = 0.8  # Heart height relative to cell size
    max_thickness: float = 0.4  # Maximum thickness (y-direction) at the top

    def condition(self, coordinate: List[float]) -> bool:
        """
        Returns True for positions that should be removed.
        Creates a heart shape with y-direction tapering.
        """
        x, y, z = coordinate

        # Center and scale coordinates
        x = (x - 0.5) * 2 / self.width
        y = (y - 0.5) * 2 / self.width
        z = (z - 0.5) * 2 / self.height

        # Heart shape equation (in xz-plane)
        heart_equation = (x**2 + z**2 - 1)**3 - x**2 * z**3
        in_heart = heart_equation <= 0  # True if within heart shape

        # Apply tapering: dynamically reduce thickness in the y-direction
        taper_factor = 1 - max(0, -z * 0.7)  # Adjust tapering intensity
        dynamic_thickness = self.max_thickness * taper_factor

        in_thickness = abs(y) <= dynamic_thickness  # Adjust thickness dynamically

        # Return True to remove atoms OUTSIDE the heart shape
        return not (in_heart and in_thickness)

# Create the condition
condition = YTaperedHeartShapeCondition(
    width=HEART_WIDTH,
    height=HEART_HEIGHT,
    max_thickness=0.4  # Adjustable max thickness
).condition


### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0]

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize(material, repetitions=[1, 1, 1], rotation="0x")
visualize(material, repetitions=[1, 1, 1], rotation="-90x")

## 2. Create target material
### 2.1. Create a slab and apply the cutout condition

In [ ]:
from mat3ra.made.tools.modify import filter_by_condition_on_coordinates, add_vacuum
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration

slab_config = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDICES,
    thickness=THICKNESS,
    vacuum=0,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX,
    use_orthogonal_z=True,
    use_conventional_cell=True,
)

slab_unit_cell = create_slab(slab_config)
resulting_material = filter_by_condition_on_coordinates(slab_unit_cell, condition, invert_selection=True)
resulting_material = add_vacuum(resulting_material, VACUUM)

## 3. Visualize the result

In [ ]:
visualize(resulting_material, repetitions=[1, 1, 1], rotation="0x")
visualize(resulting_material, repetitions=[1, 1, 1], rotation="-90x")
visualize(resulting_material, repetitions=[1, 1, 1], rotation="-90x,-90y")

# 4. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

resulting_material.name = NAME
set_materials(resulting_material)